In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV

import pandas as pd
from sqlalchemy import create_engine
import os 

# CHARGEMENT DES DONNEES 

In [ ]:
dialect = 'mysql+pymysql'
user = 'root'
password = os.getenv('mdp_mySQL')
host = 'localhost'
port = 3306
database_idf = 'SNCF_TGV_TER'

engine_url = f"{dialect}://{user}:{password}@{host}:{port}/{database_idf}"
engine = create_engine(engine_url)

In [ ]:
df_disruption = pd.read_sql('disruptions_tgv_ter',engine)
df_disruption

In [ ]:
df_vehicle = pd.read_sql('vehicle_journeys_tgv_ter', engine)
df_vehicle['vehicle_id'] = df_vehicle['vehicle_id'].apply(lambda x: ':'.join(x.split(':')[1:]))
df_vehicle

In [ ]:
df_lines = pd.read_sql_table('pt_lines_tgv_ter', engine)
df_lines

# PREPARATION DES DONNEES 

## Merge des tables 

Probleme que je ne comprend pas encore : il y a beaucoup plus de perturbation dans df_vehicle que dans df_disruption

In [ ]:
df_merge_inter = pd.merge(df_disruption.drop(['data_date','train_type'],axis=1), df_vehicle, on=['vehicle_id','id_disruption'], how='left')
df_merge = pd.merge(df_merge_inter, df_lines.drop('train_type', axis=1), on='route_id', how='left')

In [ ]:
df_merge_clean = df_merge.dropna(subset=['arrival_delay', 'departure_delay'])
df_merge_clean

### Rajout des trajets non perturbé

In [ ]:
df_vehicle_add = df_vehicle[df_vehicle['id_disruption'].isna()]
df_vehicle_add

In [ ]:
df_vehicle_add.info()

In [ ]:
df_final = pd.concat([df_merge_inter, df_vehicle_add], ignore_index=True)
df_final

# RANDOM FOREST (CLASSIFICATION)